In [42]:
from pathlib import Path
import numpy as np
import itertools
from scipy.special import logsumexp

partial = Path('./data/partial')
full = Path('./data/full')

def labelled(path):
    with open(path) as f:  
        X, Y, x, y = list(), list(), list(), list()
        for line in f:
            if line == '\n':
                X.append(x)
                Y.append(y)
                x, y = list(), list()
            else:
                word, tag = line.strip().split()
                x.append(word)
                y.append(tag)
    return X, Y

def unlabelled(path):
    with open(path) as f:  
        X, x = list(), list()
        for line in f:
            if line == '\n':
                X.append(x)
                x = list()
            else:
                word = line.strip()
                x.append(word)
    return X

def read_data(root):
    train, devin, devout = root/'train', root/'dev.in', root/'dev.out'     
    return labelled(train), unlabelled(devin), labelled(devout)

train_ds, devin_ds, devout_ds = read_data(partial)

In [8]:
def tokenize(sentence, word2index):
    return [word2index[word] if word in word2index else -1 for word in sentence]
def tag2idx(tags, tag2index):
    return [tag2index[tag] for tag in tags]

def idx_xy(X, Y, word2index=None, tag2index=None):
    if not word2index:
        vocabulary = list(set([word for sentence in X for word in sentence]))
        word2index = {word: i for i, word in enumerate(vocabulary)}
    if not tag2index:
        tags = list(set([tag for tags in Y for tag in tags]))
        tags = ['START'] + tags + ['STOP']
        tag2index = {tag: i for i, tag in enumerate(tags)}
    
    index2tag = {v:k for (k, v) in tag2index.items()}
    
    X = [tokenize(sentence, word2index) for sentence in X]
    Y = [tag2idx(tags, tag2index) for tags in Y]
    
    return X, Y, word2index, tag2index, index2tag

In [9]:
train_X, train_Y, word2index, tag2index, index2tag = idx_xy(train_ds[0], train_ds[1])
test_X, test_Y, _, _, _ = idx_xy(devout_ds[0], devout_ds[1])

In [11]:
def str_emission(x, y):
    return 'emission:' + str(y) + '+' + str(x)
def str_transition(y1, y2):
    return 'transition:' + str(y1) + '+' + str(y2)

def feature2idx(words, tags):
    features = list()
    T = len(tags)
    V = len(words)
    for y1, y2 in itertools.product(range(T), range(T)):
        features.append(str_transition(y1, y2))
    for x, y in itertools.product(range(V), range(T)):
        features.append(str_emission(x, y))
    return {f:i for i, f in enumerate(features)}

In [12]:
feature2index = feature2idx(word2index, tag2index)

In [103]:
def feature_activation(x, tag2index, feature2index):
    N, T, F = len(x), len(tag2index), len(feature2index)
    feature_activation_map = np.zeros((N+1, T, T, F), dtype=int)
    for i, y1, y2 in itertools.product(range(N+1), range(T), range(T)):
        feature_activation_map[i, y1, y2][feature2index[str_transition(y1, y2)]] = 1
        if i < N and x[i] != -1:
            feature_activation_map[i, y1, y2][feature2index[str_emission(x[i], y2)]] = 1
            
    return feature_activation_map

In [62]:
def forward(log_phi):
    """
    forward_matrix of shape (T, N+2), forward_matrix[-1, -1] is log_Z
    """
    N, T = log_phi.shape[0]-1, log_phi.shape[1]
    forward_matrix = np.ones((T, N+2))*np.NINF
    forward_matrix[0][0] = 0
    for i in range(1, N+2):
        forward_matrix[:, i] = logsumexp(np.expand_dims(forward_matrix[:, i-1], axis=1)+log_phi[i-1], axis=0)
        if i < N+1:
            forward_matrix[[0, -1], i] = np.NINF
    return forward_matrix

In [46]:
# def forward_new(log_phi):
#     """
#     forward_matrix of shape (T, N+2), forward_matrix[-1, -1] is log_Z
#     """
#     N, T = log_phi.shape[0]-1, log_phi.shape[1]
#     forward_matrix = np.ones((T, N+2))*np.NINF
#     forward_matrix[0][0] = 0
#     forward_matrix[1:-1, 1] = logsumexp(np.expand_dims(forward_matrix[:, 0], axis=1)+log_phi[0], axis=0)[1:-1]
#     for i in range(2, N+1):
#         forward_matrix[1:-1, i] = logsumexp(np.expand_dims(forward_matrix[1:-1, i-1], axis=1)+log_phi[i-1][1:-1][1:-1], axis=0)
#     forward_matrix[-1, -1] = logsumexp(np.expand_dims(forward_matrix[1:-1, i-1], axis=1)+log_phi[i-1][1:-1][-1:], axis=0)
#     return forward_matrix

In [54]:
# def forward_new(log_phi):
#     """
#     forward_matrix of shape (T, N+2), forward_matrix[-1, -1] is log_Z
#     """
#     N, T = log_phi.shape[0]-1, log_phi.shape[1]
#     forward_matrix = np.ones((T, N+2))*np.NINF
#     forward_matrix[0][0] = 0
#     for i in range(1, N+1):
#         forward_matrix[1:-1, i] = logsumexp(np.expand_dims(forward_matrix[:, i-1], axis=1)+log_phi[i-1], axis=0)[1:-1]
#     forward_matrix[-1, -1] = logsumexp(np.expand_dims(forward_matrix[:, -2], axis=1)+log_phi[-2], axis=0)[-1]
#     return forward_matrix

In [63]:
def backward(log_phi):
    """
    backward_matrix of shape (T, N+2), backward_matrix[0, 0] is log_Z
    """
    N, T = log_phi.shape[0]-1, log_phi.shape[1]
    backward_matrix = np.ones((T, N+2))*np.NINF
    backward_matrix[-1][-1] = 0
    for i in range(N, -1, -1):
        backward_matrix[:, i] = logsumexp(log_phi[i]+np.expand_dims(backward_matrix[:, i+1], axis=0), axis=1)
        if i > 0:
            backward_matrix[[0, -1], i] = np.NINF
    return backward_matrix

In [55]:
# def backward_new(log_phi):
#     """
#     backward_matrix of shape (T, N+2), backward_matrix[0, 0] is log_Z
#     """
#     N, T = log_phi.shape[0]-1, log_phi.shape[1]
#     backward_matrix = np.ones((T, N+2))*np.NINF
#     backward_matrix[-1][-1] = 0
#     for i in range(N, 0, -1):
#         backward_matrix[1:-1, i] = logsumexp(log_phi[i]+np.expand_dims(backward_matrix[:, i+1], axis=0), axis=1)[1:-1]
#     backward_matrix[0, 0] = logsumexp(log_phi[0]+np.expand_dims(backward_matrix[:, 1], axis=0), axis=1)[0]
#     return backward_matrix

In [100]:
np.random.seed(1)

theta = np.random.randn(len(feature2index))
feature_activation_map = feature_activation(train_X[0], tag2index, feature2index)
log_phi = np.dot(feature_activation_map, theta)

forward_matrix = forward(log_phi)
backward_matrix = backward(log_phi)

forward_matrix[-1, -1], backward_matrix[0][0]

(88.83919398772295, 88.83919398772295)

In [101]:
np.random.seed(1)

theta = np.random.randn(len(feature2index))
feature_activation_map = feature_activation(train_X[0], tag2index, feature2index)
log_phi = np.dot(feature_activation_map, theta)

forward_matrix = forward_new(log_phi)
backward_matrix = backward_new(log_phi)

forward_matrix[-1, -1], backward_matrix[0][0]

(88.59832225452645, 88.83919398772295)

In [98]:
def NLL(theta, feature2index, X, Y, tag2index, param=0.1):
    log_likelihood = 0
    gradient = np.zeros(len(theta))
    exp_features = np.zeros(len(theta))
    emp_features = np.zeros(len(theta))
    T = len(tag2index)
    counter = 1

    for x, y in zip(X, Y):
        N = len(x)

        activation_map = feature_activation(x, tag2index, feature2index)
        log_phi = np.dot(activation_map, theta)

        alpha = forward(log_phi)
        beta = backward(log_phi)

        log_Z = beta[0][0]

        log_likelihood += log_phi[0, 0, y[0]]
        emp_features += activation_map[0, 0, y[0]]

        for y2 in range(1, T-1): 
            exp_features += np.exp(alpha[0, 0] + log_phi[0, 0, y2] + beta[y2, 1] - log_Z) *\
                            activation_map[0, 0, y2]

        for i in range(1, N):
            emp_features += activation_map[i, y[i-1], y[i]]
            log_likelihood += log_phi[i, y[i-1], y[i]]

            for y1, y2 in itertools.product(range(1, T-1), range(1, T-1)):
                exp_features += np.exp(alpha[y1, i] + log_phi[i, y1, y2] + beta[y2, i+1] - log_Z) *\
                                activation_map[i, y1, y2]

        for y1 in range(1, T-1):
            exp_features += np.exp(alpha[-1, N] + log_phi[N, y1, -1] + beta[-1, N+1] - log_Z) *\
                            activation_map[N, y1, -1]

        emp_features += activation_map[N, y[-1], -1]

        log_likelihood += log_phi[N, y[-1], -1]

        log_likelihood -= log_Z
        
        print('done with the {}th instance'.format(counter))
        counter += 1
        
    gradient = exp_features - emp_features
    
    loss = -log_likelihood + param*(np.sum(theta**2))
    
    gradient += param*2*theta
    
    return loss, gradient

In [102]:
def viterbi(X, tag2index, feature2index, theta):
    
    Y = []
    index2tag = {v-1: k for k, v in tag2index.items()}

    for x in X:
        activation_map = feature_activation(x, tag2index, feature2index)
        log_phi = np.dot(activation_map, theta)
        N, T = len(x), len(tag2index)
        score = np.zeros((T-2, N), dtype=np.float)
        path = np.zeros((T-2, N), dtype=np.int)
        
        score[:, 0] = log_phi[0, 0, 1:-1]
        for i in range(1, N):
            cross = score[:, i-1][:, None] + log_phi[i, 1:-1, 1:-1]
            score[:, i] = np.max(cross, axis=0)
            path[:, i] = np.argmax(cross, axis=0)

        y = []
        last_tag = np.argmax(score[:, -1] + log_phi[-1, 1:-1, -1])
        ybest = last_tag

        for i in range(len(x)-1,-1,-1):
            y.insert(0, ybest)
            ybest = path[i, ybest]

        Y.append([index2tag[idx] for idx in y])

    return Y

In [105]:
from scipy.optimize import fmin_l_bfgs_b

def callbackF(w):
    loss = get_loss_grad(w)[0]
    print('Loss:{0:.4f}'.format(loss))
    
def get_loss_grad(w):
    loss, grads = NLL(w, feature2index, train_X[:2], train_Y[:2], tag2index)
    return loss, grads

init_w = np.zeros(len(feature2index))
result = fmin_l_bfgs_b(get_loss_grad, init_w, pgtol=0.01, callback=callbackF)

done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
Loss:12.1793
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
Loss:7.0047
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
Loss:6.3043
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
Loss:5.7807
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done with the 2th instance
done with the 1th instance
done w